# Setup

### Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
import time
from utility import *
from tqdm import trange

from epnp import EPnP

# Sets 
np.set_printoptions(formatter={'float': '{: 0.2f}'.format})

# Removes warnings in command line when using laptop
o3d.utility.set_verbosity_level(o3d.cpu.pybind.utility.VerbosityLevel.Error)

### Paths to CAD files

In [ ]:
# As far as I know, jupyter notebooks require absolute paths
abs_path = "D:\\Documents\\1_Skole\\NTNU\\Semester_11\\Master\\Master\\" # laptop
# abs_path = "C:\\Users\\Runar\\Documents\\Skole\\NTNU\\Semester_11\\Master\\Code\\" # desktop

path_to_cad_car_06          = abs_path + "CAD\\car\\06.off"
path_to_cad_bicycle_01      = abs_path + "CAD\\bicycle\\01.off" 
path_to_cad_motorbike_04    = abs_path + "CAD\\motorbike\\04.off" 
path_to_cad_sofa_02         = abs_path + "CAD\\sofa\\02.off" 
path_to_cad_bus_06          = abs_path + "CAD\\bus\\06.off" 

### Tests to do

In [ ]:
# Just variables to change what is tested / Change to True if you want to test something
randtrans   = True
onlytrans   = False
small_rot   = False
init_gnc    = False
rt_eqiv     = False
varying_n   = False

# Random Transformation

In [ ]:
if randtrans == 1: 
    print(f"Random Transformations")

    # Experiment Setup
    test_outlier = 98+1
    test_amount  = 20
    
    # Point Cloud
    ph_CAD = load_points_from_file(path_to_cad_car_06)
    ph_CAD = downsample_points(ph_CAD, 100)
    ph_CAD = scale_points(ph_CAD, 10)

    # Camera Parameter Matrix
    focal = 800.
    u_0 = 320.
    v_0 = 240.
    Ca =  compute_C(focal, focal, u_0, v_0)

    # Test Loop
    list_randtrans = []
    for i in trange(0, test_outlier):
        list_randtrans.append([])
        for j in range(test_amount):
            Tr_random = compute_T(
                        np.random.uniform(-np.pi, np.pi),   # x-rotation
                        np.random.uniform(-np.pi, np.pi),   # y-rotation
                        np.random.uniform(-np.pi, np.pi),   # z-rotation 
                        np.random.uniform(-0.5,0.5),            # x-translation
                        np.random.uniform(-0.5,0.5),            # x-translation
                        np.random.uniform(-0.5,0.5)*3+10)       # x-translation
            pix = compute_pixels(ph_CAD, Tr_random, Ca, sigma=5, outlier_percentage=i)
            pix, ph_CAD = shuffle_points(pix, ph_CAD)

            epnp = EPnP()
            epnp.load_data(ph_CAD, pix, Tr_random, Ca)
            epnp.define_gnc_parameters(eps = 1000)
            epnp.compute_epnp(GN = True, GNC=True)
            list_randtrans[i].append(epnp)

#### Plotting

In [ ]:
if randtrans == True:
    fig, ax2 = plt.subplots(2,5, figsize=(20, 8))
    # fig.suptitle("Random transformation")
    # test_outlier = 62

    xlim = [0, test_outlier]
    ylim = [-5, 185]


    outlier_count = [x for x in range(test_outlier)]

    # Pre opt
    rot_err_all = []
    trans_err_all = []
    for i in range(test_outlier):
        rot_err = []
        tra_err = []
        temp1 = list_randtrans[i]
        for j in range(test_amount):
            temp2 = temp1[j]
            rot_err.append(angular_distance_mat(temp2.T[:3,:3], temp2.Rt_pre_opt[:3,:3]))
            tra_err.append(translation_error(temp2.T[:3,3], temp2.Rt_pre_opt[:3,3]))
        rot_err_all.append(rot_err)
        trans_err_all.append(tra_err)
    # Rotation
    ax2[0, 0].boxplot(rot_err_all, labels=outlier_count)
    ax2[0, 0].set_title("EPnP")
    ax2[0, 0].set_xlim(xlim)
    ax2[0, 0].set_ylim(ylim)
    # Translation
    ax2[1, 0].boxplot(trans_err_all, labels=outlier_count)
    ax2[1, 0].set_xlim(xlim)
    ax2[1, 0].set_ylim(ylim)

    # GN
    rot_err_all = []
    trans_err_all = []
    for i in range(test_outlier):
        rot_err = []
        tra_err = []
        temp1 = list_randtrans[i]
        for j in range(test_amount):
            temp2 = temp1[j]
            rot_err.append(angular_distance_mat(temp2.T[:3,:3], temp2.Rt_CV_EPnP[:3,:3]))
            tra_err.append(translation_error(temp2.T[:3,3], temp2.Rt_CV_EPnP[:3,3]))
        rot_err_all.append(rot_err)
        trans_err_all.append(tra_err)
    ax2[0, 1].boxplot(rot_err_all, labels=outlier_count)
    ax2[0, 1].set_title("OpenCV EPnP")
    ax2[0, 1].set_xlim(xlim)
    ax2[0, 1].set_ylim(ylim)

    ax2[1, 1].boxplot(trans_err_all, labels=outlier_count)
    ax2[1, 1].set_xlim(xlim)
    ax2[1, 1].set_ylim(ylim)

    # GNC
    rot_err_all = []
    trans_err_all = []
    for i in range(test_outlier):
        rot_err = []
        tra_err = []
        temp1 = list_randtrans[i]
        for j in range(test_amount):
            temp2 = temp1[j]
            rot_err.append(angular_distance_mat(temp2.T[:3,:3], temp2.Rt_CV_SQPnP[:3,:3]))
            tra_err.append(translation_error(temp2.T[:3,3], temp2.Rt_CV_SQPnP[:3,3]))
        rot_err_all.append(rot_err)
        trans_err_all.append(tra_err)
    ax2[0, 2].boxplot(rot_err_all, labels=outlier_count)
    ax2[0, 2].set_title("OpenCV SQPnP")
    ax2[0, 2].set_xlim(xlim)
    ax2[0, 2].set_ylim(ylim)

    ax2[1, 2].boxplot(trans_err_all, labels=outlier_count)
    ax2[1, 2].set_xlim(xlim)
    ax2[1, 2].set_ylim(ylim)

    # OpenCV EPnP
    rot_err_all = []
    trans_err_all = []
    for i in range(test_outlier):
        rot_err = []
        tra_err = []
        temp1 = list_randtrans[i]
        for j in range(test_amount):
            temp2 = temp1[j]
            rot_err.append(angular_distance_mat(temp2.T[:3,:3], temp2.Rt_CV_Ransac[:3,:3]))
            tra_err.append(translation_error(temp2.T[:3,3], temp2.Rt_CV_Ransac[:3,3]))
        rot_err_all.append(rot_err)
        trans_err_all.append(tra_err)
    ax2[0, 3].boxplot(rot_err_all, labels=outlier_count)
    ax2[0, 3].set_title("OpenCV RANSAC")
    ax2[0, 3].set_xlim(xlim)
    ax2[0, 3].set_ylim(ylim)

    ax2[1, 3].boxplot(trans_err_all, labels=outlier_count)
    ax2[1, 3].set_xlim(xlim)
    ax2[1, 3].set_ylim(ylim)

    # OpenCV RANSAC
    rot_err_all = []
    trans_err_all = []
    for i in range(test_outlier):
        rot_err = []
        tra_err = []
        temp1 = list_randtrans[i]
        for j in range(test_amount):
            temp2 = temp1[j]
            rot_err.append(angular_distance_mat(temp2.T[:3,:3], temp2.Rt_best[:3,:3]))
            tra_err.append(translation_error(temp2.T[:3,3], temp2.Rt_best[:3,3]))
        rot_err_all.append(rot_err)
        trans_err_all.append(tra_err)
    ax2[0, 4].boxplot(rot_err_all, labels=outlier_count)
    ax2[0, 4].set_title("EPnP + GNC")
    ax2[0, 4].set_xlim(xlim)
    ax2[0, 4].set_ylim(ylim)

    ax2[1, 4].boxplot(trans_err_all, labels=outlier_count)
    ax2[1, 4].set_xlim(xlim)
    ax2[1, 4].set_ylim(ylim)

    plt.setp(ax2[0, 0], ylabel='Rotation error')
    plt.setp(ax2[1, 0], ylabel='Translation error')

    for i, a in enumerate(ax2):
        for j, b in enumerate(a):
            for n, label in enumerate(b.xaxis.get_ticklabels()):
                if (n) % 10 != 0:
                    label.set_visible(False)

    plt.show()

In [ ]:
if randtrans == True:
    fig, ax2 = plt.subplots(1,5, figsize=(20, 4))
    # fig.suptitle("Random transformation")
    # test_outlier = 62

    xlim = [0, test_outlier]
    ylim = [0, 2]


    outlier_count = [x for x in range(test_outlier)]

    # Pre opt
    rot_err_all = []
    for i in range(test_outlier):
        rot_err = []
        temp1 = list_randtrans[i]
        for j in range(test_amount):
            temp2 = temp1[j]
            rot_err.append(temp2.timing_GN*0.000000001)
        rot_err_all.append(rot_err)
    # Rotation
    ax2[0].boxplot(rot_err_all, labels=outlier_count)
    ax2[0].set_title("EPnP")
    ax2[0].set_xlim(xlim)
    ax2[0].set_ylim([0,0.06])

    # GN
    rot_err_all = []
    for i in range(test_outlier):
        rot_err = []
        temp1 = list_randtrans[i]
        for j in range(test_amount):
            temp2 = temp1[j]
            rot_err.append(temp2.timing_opencv_epnp*0.000000001)
        rot_err_all.append(rot_err)
    ax2[1].boxplot(rot_err_all, labels=outlier_count)
    ax2[1].set_title("OpenCV EPnP")
    ax2[1].set_xlim(xlim)
    ax2[1].set_ylim([0,0.005])

    # GNC
    rot_err_all = []
    for i in range(test_outlier):
        rot_err = []
        temp1 = list_randtrans[i]
        for j in range(test_amount):
            temp2 = temp1[j]
            rot_err.append(temp2.timing_opencv_sqpnp*0.000000001)
        rot_err_all.append(rot_err)
    ax2[2].boxplot(rot_err_all, labels=outlier_count)
    ax2[2].set_title("OpenCV SQPnP")
    ax2[2].set_xlim(xlim)
    ax2[2].set_ylim([0,0.005])

    # OpenCV EPnP
    rot_err_all = []
    for i in range(test_outlier):
        rot_err = []
        temp1 = list_randtrans[i]
        for j in range(test_amount):
            temp2 = temp1[j]
            rot_err.append(temp2.timing_opencv_ransac*0.000000001)
        rot_err_all.append(rot_err)
    ax2[3].boxplot(rot_err_all, labels=outlier_count)
    ax2[3].set_title("OpenCV RANSAC")
    ax2[3].set_xlim(xlim)
    ax2[3].set_ylim([0,0.02])

    # OpenCV RANSAC
    rot_err_all = []
    for i in range(test_outlier):
        rot_err = []
        temp1 = list_randtrans[i]
        for j in range(test_amount):
            temp2 = temp1[j]
            rot_err.append(temp2.timing_GNC*0.000000001)
        rot_err_all.append(rot_err)
    ax2[4].boxplot(rot_err_all, labels=outlier_count)
    ax2[4].set_title("EPnP + GNC")
    ax2[4].set_xlim(xlim)
    ax2[4].set_ylim([0,1.2])

    plt.setp(ax2[0], ylabel='seconds')
    plt.setp(ax2[:], xlabel='outlier percentage')

    for i, a in enumerate(ax2):
        # a.grid()
        for n, label in enumerate(a.xaxis.get_ticklabels()):
            if (n) % 10 != 0:
                label.set_visible(False)

    plt.show()

# Only Translation

In [ ]:
if onlytrans == 1: 
    print(f"Only Translation, No Rotation")

    # Experiment Setup
    test_outlier = 98+1
    test_amount  = 20
    
    # Point Cloud
    ph_CAD = load_points_from_file(path_to_cad_car_06)
    ph_CAD = downsample_points(ph_CAD, 100)
    ph_CAD = scale_points(ph_CAD, 10)

    # Transformation Matrices
    Tr_onlytrans = compute_T(0, 0, 0,     0, 0, 10)

    # Camera Parameter Matrix
    focal = 800.
    u_0 = 320.
    v_0 = 240.
    Ca =  compute_C(focal, focal, u_0, v_0)

    # Test Loop
    list_onlytrans = []
    for i in trange(0, test_outlier):
        list_onlytrans.append([])
        for j in range(test_amount):
            pix = compute_pixels(ph_CAD, Tr_onlytrans, Ca, sigma=5, outlier_percentage=i)
            pix, ph_CAD = shuffle_points(pix, ph_CAD)

            epnp = EPnP()
            epnp.load_data(ph_CAD, pix, Tr_onlytrans, Ca)
            epnp.define_gnc_parameters(eps = 1000)
            epnp.compute_epnp(GN = True, GNC=True)
            list_onlytrans[i].append(epnp)

#### Plotting

In [ ]:
if onlytrans == True:
    fig, ax2 = plt.subplots(2,5, figsize=(20, 8))
    # test_outlier = 62

    xlim = [0, test_outlier]
    ylim = [-5, 181]


    outlier_count = [x for x in range(test_outlier)]

    # Pre opt
    rot_err_all = []
    trans_err_all = []
    for i in range(test_outlier):
        rot_err = []
        tra_err = []
        temp1 = list_onlytrans[i]
        for j in range(test_amount):
            temp2 = temp1[j]
            rot_err.append(angular_distance_mat(temp2.T[:3,:3], temp2.Rt_pre_opt[:3,:3]))
            tra_err.append(translation_error(temp2.T[:3,3], temp2.Rt_pre_opt[:3,3]))
        rot_err_all.append(rot_err)
        trans_err_all.append(tra_err)
    # Rotation
    ax2[0, 0].boxplot(rot_err_all, labels=outlier_count)
    ax2[0, 0].set_title("EPnP")
    ax2[0, 0].set_xlim(xlim)
    ax2[0, 0].set_ylim(ylim)
    # Translation
    ax2[1, 0].boxplot(trans_err_all, labels=outlier_count)
    ax2[1, 0].set_xlim(xlim)
    ax2[1, 0].set_ylim(ylim)

    # GN
    rot_err_all = []
    trans_err_all = []
    for i in range(test_outlier):
        rot_err = []
        tra_err = []
        temp1 = list_onlytrans[i]
        for j in range(test_amount):
            temp2 = temp1[j]
            rot_err.append(angular_distance_mat(temp2.T[:3,:3], temp2.Rt_CV_EPnP[:3,:3]))
            tra_err.append(translation_error(temp2.T[:3,3], temp2.Rt_CV_EPnP[:3,3]))
        rot_err_all.append(rot_err)
        trans_err_all.append(tra_err)
    ax2[0, 1].boxplot(rot_err_all, labels=outlier_count)
    ax2[0, 1].set_title("OpenCV EPnP")
    ax2[0, 1].set_xlim(xlim)
    ax2[0, 1].set_ylim(ylim)

    ax2[1, 1].boxplot(trans_err_all, labels=outlier_count)
    ax2[1, 1].set_xlim(xlim)
    ax2[1, 1].set_ylim(ylim)

    # GNC
    rot_err_all = []
    trans_err_all = []
    for i in range(test_outlier):
        rot_err = []
        tra_err = []
        temp1 = list_onlytrans[i]
        for j in range(test_amount):
            temp2 = temp1[j]
            rot_err.append(angular_distance_mat(temp2.T[:3,:3], temp2.Rt_CV_SQPnP[:3,:3]))
            tra_err.append(translation_error(temp2.T[:3,3], temp2.Rt_CV_SQPnP[:3,3]))
        rot_err_all.append(rot_err)
        trans_err_all.append(tra_err)
    ax2[0, 2].boxplot(rot_err_all, labels=outlier_count)
    ax2[0, 2].set_title("OpenCV SQPnP")
    ax2[0, 2].set_xlim(xlim)
    ax2[0, 2].set_ylim(ylim)

    ax2[1, 2].boxplot(trans_err_all, labels=outlier_count)
    ax2[1, 2].set_xlim(xlim)
    ax2[1, 2].set_ylim(ylim)

    # OpenCV EPnP
    rot_err_all = []
    trans_err_all = []
    for i in range(test_outlier):
        rot_err = []
        tra_err = []
        temp1 = list_onlytrans[i]
        for j in range(test_amount):
            temp2 = temp1[j]
            rot_err.append(angular_distance_mat(temp2.T[:3,:3], temp2.Rt_CV_Ransac[:3,:3]))
            tra_err.append(translation_error(temp2.T[:3,3], temp2.Rt_CV_Ransac[:3,3]))
        rot_err_all.append(rot_err)
        trans_err_all.append(tra_err)
    ax2[0, 3].boxplot(rot_err_all, labels=outlier_count)
    ax2[0, 3].set_title("OpenCV RANSAC")
    ax2[0, 3].set_xlim(xlim)
    ax2[0, 3].set_ylim(ylim)

    ax2[1, 3].boxplot(trans_err_all, labels=outlier_count)
    ax2[1, 3].set_xlim(xlim)
    ax2[1, 3].set_ylim(ylim)

    # OpenCV RANSAC
    rot_err_all = []
    trans_err_all = []
    for i in range(test_outlier):
        rot_err = []
        tra_err = []
        temp1 = list_onlytrans[i]
        for j in range(test_amount):
            temp2 = temp1[j]
            rot_err.append(angular_distance_mat(temp2.T[:3,:3], temp2.Rt_best[:3,:3]))
            tra_err.append(translation_error(temp2.T[:3,3], temp2.Rt_best[:3,3]))
        rot_err_all.append(rot_err)
        trans_err_all.append(tra_err)
    ax2[0, 4].boxplot(rot_err_all, labels=outlier_count)
    ax2[0, 4].set_title("EPnP + GNC")
    ax2[0, 4].set_xlim(xlim)
    ax2[0, 4].set_ylim(ylim)

    ax2[1, 4].boxplot(trans_err_all, labels=outlier_count)
    ax2[1, 4].set_xlim(xlim)
    ax2[1, 4].set_ylim(ylim)

    plt.setp(ax2[0, 0], ylabel='Rotation error')
    plt.setp(ax2[1, 0], ylabel='Translation error')

    for i, a in enumerate(ax2):
        for j, b in enumerate(a):
            for n, label in enumerate(b.xaxis.get_ticklabels()):
                if (n) % 10 != 0:
                    label.set_visible(False)

    plt.show()

# Small Rotations

In [ ]:
if small_rot == 1:
    print(f"Small rotations")

    # Experiment Setup
    test_outlier = 98+1
    test_amount  = 20
    
    # Point Cloud
    ph_CAD = load_points_from_file(path_to_cad_car_06)
    ph_CAD = downsample_points(ph_CAD, 100)
    ph_CAD = scale_points(ph_CAD, 10)

    # Camera Parameter Matrix
    focal = 800.
    u_0 = 320.
    v_0 = 240.
    Ca =  compute_C(focal, focal, u_0, v_0)

    # Test Loop
    list_small_rot = []
    for i in trange(0, test_outlier):
        list_small_rot.append([])
        for j in range(test_amount):
            Tr_random = compute_T(
                        np.random.uniform(-np.pi/20, np.pi/20),   # x-rotation
                        np.random.uniform(-np.pi/20, np.pi/20),   # y-rotation
                        np.random.uniform(-np.pi/20, np.pi/20),   # z-rotation 
                        np.random.uniform(-0.5,0.5),        # x-translation
                        np.random.uniform(-0.5,0.5),        # x-translation
                        np.random.uniform(-0.5,0.5)*3+10)  # x-translation
            pix = compute_pixels(ph_CAD, Tr_random, Ca, sigma=5, outlier_percentage=i)
            pix, ph_CAD = shuffle_points(pix, ph_CAD)

            epnp = EPnP()
            epnp.load_data(ph_CAD, pix, Tr_random, Ca)
            epnp.define_gnc_parameters(eps = 1000)
            epnp.compute_epnp(GN = True, GNC=True)
            list_small_rot[i].append(epnp)

#### Plotting

In [ ]:
if small_rot == True:
    fig, ax2 = plt.subplots(2,5, figsize=(20, 8))
    # fig.suptitle("Random transformation")
    # test_outlier = 62

    xlim = [0, test_outlier]
    ylim = [-5, 185]


    outlier_count = [x for x in range(test_outlier)]

    # Pre opt
    rot_err_all = []
    trans_err_all = []
    for i in range(test_outlier):
        rot_err = []
        tra_err = []
        temp1 = list_small_rot[i]
        for j in range(test_amount):
            temp2 = temp1[j]
            rot_err.append(angular_distance_mat(temp2.T[:3,:3], temp2.Rt_pre_opt[:3,:3]))
            tra_err.append(translation_error(temp2.T[:3,3], temp2.Rt_pre_opt[:3,3]))
        rot_err_all.append(rot_err)
        trans_err_all.append(tra_err)
    # Rotation
    ax2[0, 0].boxplot(rot_err_all, labels=outlier_count)
    ax2[0, 0].set_title("EPnP")
    ax2[0, 0].set_xlim(xlim)
    ax2[0, 0].set_ylim(ylim)
    # Translation
    ax2[1, 0].boxplot(trans_err_all, labels=outlier_count)
    ax2[1, 0].set_xlim(xlim)
    ax2[1, 0].set_ylim(ylim)

    # GN
    rot_err_all = []
    trans_err_all = []
    for i in range(test_outlier):
        rot_err = []
        tra_err = []
        temp1 = list_small_rot[i]
        for j in range(test_amount):
            temp2 = temp1[j]
            rot_err.append(angular_distance_mat(temp2.T[:3,:3], temp2.Rt_CV_EPnP[:3,:3]))
            tra_err.append(translation_error(temp2.T[:3,3], temp2.Rt_CV_EPnP[:3,3]))
        rot_err_all.append(rot_err)
        trans_err_all.append(tra_err)
    ax2[0, 1].boxplot(rot_err_all, labels=outlier_count)
    ax2[0, 1].set_title("OpenCV EPnP")
    ax2[0, 1].set_xlim(xlim)
    ax2[0, 1].set_ylim(ylim)

    ax2[1, 1].boxplot(trans_err_all, labels=outlier_count)
    ax2[1, 1].set_xlim(xlim)
    ax2[1, 1].set_ylim(ylim)

    # GNC
    rot_err_all = []
    trans_err_all = []
    for i in range(test_outlier):
        rot_err = []
        tra_err = []
        temp1 = list_small_rot[i]
        for j in range(test_amount):
            temp2 = temp1[j]
            rot_err.append(angular_distance_mat(temp2.T[:3,:3], temp2.Rt_CV_SQPnP[:3,:3]))
            tra_err.append(translation_error(temp2.T[:3,3], temp2.Rt_CV_SQPnP[:3,3]))
        rot_err_all.append(rot_err)
        trans_err_all.append(tra_err)
    ax2[0, 2].boxplot(rot_err_all, labels=outlier_count)
    ax2[0, 2].set_title("OpenCV SQPnP")
    ax2[0, 2].set_xlim(xlim)
    ax2[0, 2].set_ylim(ylim)

    ax2[1, 2].boxplot(trans_err_all, labels=outlier_count)
    ax2[1, 2].set_xlim(xlim)
    ax2[1, 2].set_ylim(ylim)

    # OpenCV EPnP
    rot_err_all = []
    trans_err_all = []
    for i in range(test_outlier):
        rot_err = []
        tra_err = []
        temp1 = list_small_rot[i]
        for j in range(test_amount):
            temp2 = temp1[j]
            rot_err.append(angular_distance_mat(temp2.T[:3,:3], temp2.Rt_CV_Ransac[:3,:3]))
            tra_err.append(translation_error(temp2.T[:3,3], temp2.Rt_CV_Ransac[:3,3]))
        rot_err_all.append(rot_err)
        trans_err_all.append(tra_err)
    ax2[0, 3].boxplot(rot_err_all, labels=outlier_count)
    ax2[0, 3].set_title("OpenCV RANSAC")
    ax2[0, 3].set_xlim(xlim)
    ax2[0, 3].set_ylim(ylim)

    ax2[1, 3].boxplot(trans_err_all, labels=outlier_count)
    ax2[1, 3].set_xlim(xlim)
    ax2[1, 3].set_ylim(ylim)

    # OpenCV RANSAC
    rot_err_all = []
    trans_err_all = []
    for i in range(test_outlier):
        rot_err = []
        tra_err = []
        temp1 = list_small_rot[i]
        for j in range(test_amount):
            temp2 = temp1[j]
            rot_err.append(angular_distance_mat(temp2.T[:3,:3], temp2.Rt_best[:3,:3]))
            tra_err.append(translation_error(temp2.T[:3,3], temp2.Rt_best[:3,3]))
        rot_err_all.append(rot_err)
        trans_err_all.append(tra_err)
    ax2[0, 4].boxplot(rot_err_all, labels=outlier_count)
    ax2[0, 4].set_title("EPnP + GNC")
    ax2[0, 4].set_xlim(xlim)
    ax2[0, 4].set_ylim(ylim)

    ax2[1, 4].boxplot(trans_err_all, labels=outlier_count)
    ax2[1, 4].set_xlim(xlim)
    ax2[1, 4].set_ylim(ylim)

    plt.setp(ax2[0, 0], ylabel='Rotation error')
    plt.setp(ax2[1, 0], ylabel='Translation error')

    for i, a in enumerate(ax2):
        for j, b in enumerate(a):
            for n, label in enumerate(b.xaxis.get_ticklabels()):
                if (n) % 10 != 0:
                    label.set_visible(False)

    plt.show()

# Different Initializations of GNC

In [ ]:
# This experiment needs to be done on its own
if init_gnc == 1:
    print(f"Different initializations of GNC")

    # Experiment Setup
    test_outlier = 98+1
    test_amount  = 20
    
    # Point Cloud
    ph_CAD = load_points_from_file(path_to_cad_car_06)
    ph_CAD = downsample_points(ph_CAD, 100)
    ph_CAD = scale_points(ph_CAD, 10)

    # Transformation Matrices
    # This is calculated in each iteration

    # Camera Parameter Matrix
    focal = 800.
    u_0 = 320.
    v_0 = 240.
    Ca =  compute_C(focal, focal, u_0, v_0)

    list_init = []
    num_per_init = 4
    for i in range(num_per_init): list_init.append("Eye")
    for i in range(num_per_init): list_init.append("EPnP")
    for i in range(num_per_init): list_init.append("OpenCV-EPnP")
    for i in range(num_per_init): list_init.append("Trans")
    

    # Test Loop
    list_gnc_init = []
    for i in trange(0, test_outlier):
        list_gnc_init.append([])
        for j in range(len(list_init)):
            Tr_random = compute_T(
                        np.random.uniform(-np.pi/2, np.pi/2),   # x-rotation
                        np.random.uniform(-np.pi/2, np.pi/2),   # y-rotation
                        np.random.uniform(-np.pi/2, np.pi/2),   # z-rotation 
                        np.random.uniform(-0.5,0.5),        # x-translation
                        np.random.uniform(-0.5,0.5),        # x-translation
                        np.random.uniform(-0.5,0.5)*3+10)  # x-translation

            pix = compute_pixels(ph_CAD, Tr_random, Ca, sigma=5, outlier_percentage=i)
            pix, ph_CAD = shuffle_points(pix, ph_CAD)
            
            epnp = EPnP()
            epnp.load_data(ph_CAD, pix, Tr_random, Ca)
            epnp.define_gnc_parameters(eps = 1000)
            epnp.Rt_choose = list_init[j]
            epnp.compute_epnp(GN = True, GNC=True)
            list_gnc_init[i].append(epnp)

#### Plotting

In [ ]:
if init_gnc == 1:
    fig, ax2 = plt.subplots(2,4, figsize=(16, 8))
    # test_outlier = 62

    xlim = [0, test_outlier]
    ylim = [-5, 181]


    outlier_count = [x for x in range(test_outlier)]

    # Eye
    rot_err_eye = []
    trans_err_eye = []
    for i in range(test_outlier):
        temp_rt = list_gnc_init[i]
        temp_rot = []
        temp_trans=[]
        for j in range(4):
            temp_rot.append(angular_distance_mat(temp_rt[0*num_per_init+j].T[:3,:3], temp_rt[0*num_per_init+j].Rt_best[:3,:3]))
            temp_trans.append(translation_error(temp_rt[0*num_per_init+j].T[:3,3], temp_rt[0*num_per_init+j].Rt_best[:3,3]))
        rot_err_eye.append(temp_rot)
        trans_err_eye.append(temp_trans)
    # Rotation
    ax2[0, 0].boxplot(rot_err_eye, labels=outlier_count)
    ax2[0, 0].set_title("Eye")
    ax2[0, 0].set_xlim(xlim)
    ax2[0, 0].set_ylim(ylim)
    # Translation
    ax2[1, 0].boxplot(trans_err_eye, labels=outlier_count)
    ax2[1, 0].set_xlim(xlim)
    ax2[1, 0].set_ylim(ylim)

    # EPnP
    rot_err_epnp = []
    trans_err_epnp = []
    for i in range(test_outlier):
        temp_rt = list_gnc_init[i]
        temp_rot = []
        temp_trans=[]
        for j in range(4):
            temp_rot.append(angular_distance_mat(temp_rt[1*num_per_init+j].T[:3,:3], temp_rt[1*num_per_init+j].Rt_best[:3,:3]))
            temp_trans.append(translation_error(temp_rt[1*num_per_init+j].T[:3,3], temp_rt[1*num_per_init+j].Rt_best[:3,3]))
        rot_err_epnp.append(temp_rot)
        trans_err_epnp.append(temp_trans)
    ax2[0, 1].boxplot(rot_err_epnp, labels=outlier_count)
    ax2[0, 1].set_title("EPnP")
    ax2[0, 1].set_xlim(xlim)
    ax2[0, 1].set_ylim(ylim)
    # Translation
    ax2[1, 1].boxplot(trans_err_epnp, labels=outlier_count)
    ax2[1, 1].set_xlim(xlim)
    ax2[1, 1].set_ylim(ylim)

    # OpenCV-EPnP+GN
    rot_err_opencv = []
    trans_err_opencv = []
    for i in range(test_outlier):
        temp_rt = list_gnc_init[i]
        temp_rot = []
        temp_trans=[]
        for j in range(4):
            temp_rot.append(angular_distance_mat(temp_rt[2*num_per_init+j].T[:3,:3], temp_rt[2*num_per_init+j].Rt_best[:3,:3]))
            temp_trans.append(translation_error(temp_rt[2*num_per_init+j].T[:3,3], temp_rt[2*num_per_init+j].Rt_best[:3,3]))
        rot_err_opencv.append(temp_rot)
        trans_err_opencv.append(temp_trans)
    ax2[0, 2].boxplot(rot_err_opencv, labels=outlier_count)
    ax2[0, 2].set_title("OpenCV EPnP+GN")
    ax2[0, 2].set_xlim(xlim)
    ax2[0, 2].set_ylim(ylim)
    ax2[1, 2].boxplot(trans_err_opencv, labels=outlier_count)
    ax2[1, 2].set_xlim(xlim)
    ax2[1, 2].set_ylim(ylim)

    # OpenCV-EPnP+GN
    rot_err_trans_init = []
    trans_err_trans_init = []
    for i in range(test_outlier):
        temp_rt = list_gnc_init[i]
        temp_rot = []
        temp_trans=[]
        for j in range(4):
            temp_rot.append(angular_distance_mat(temp_rt[3*num_per_init+j].T[:3,:3], temp_rt[3*num_per_init+j].Rt_best[:3,:3]))
            temp_trans.append(translation_error(temp_rt[3*num_per_init+j].T[:3,3], temp_rt[3*num_per_init+j].Rt_best[:3,3]))
        rot_err_trans_init.append(temp_rot)
        trans_err_trans_init.append(temp_trans)
    ax2[0, 3].boxplot(rot_err_trans_init, labels=outlier_count)
    ax2[0, 3].set_title("Small Translation")
    ax2[0, 3].set_xlim(xlim)
    ax2[0, 3].set_ylim(ylim)
    ax2[1, 3].boxplot(trans_err_trans_init, labels=outlier_count)
    ax2[1, 3].set_xlim(xlim)
    ax2[1, 3].set_ylim(ylim)

    plt.setp(ax2[0, 0], ylabel='Rotation error')
    plt.setp(ax2[1, 0], ylabel='Translation error')

    for i, a in enumerate(ax2):
        for j, b in enumerate(a):
            for n, label in enumerate(b.xaxis.get_ticklabels()):
                if (n) % 10 != 0:
                    label.set_visible(False)

    plt.show()

In [ ]:
# Plotting time and iterations
if init_gnc == 1:
    fig, ax2 = plt.subplots(2,4, figsize=(16, 8))
    # test_outlier = 62

    xlim = [0, test_outlier]
    ylim_time = [ 0,   3]
    ylim_iter = [ 0, 250]

    outlier_count = [x for x in range(test_outlier)]

    # Eye
    rot_err_eye = []
    trans_err_eye = []
    for i in range(test_outlier):
        temp_rt = list_gnc_init[i]
        temp_rot = []
        temp_trans=[]
        for j in range(4):
            temp_rot.append(temp_rt[j].timing_GNC)
            temp_trans.append(temp_rt[j].iterations)
        rot_err_eye.append(temp_rot)
        trans_err_eye.append(temp_trans)
    # Rotation
    ax2[0, 0].boxplot(rot_err_eye, labels=outlier_count)
    ax2[0, 0].set_title("Eye")
    ax2[0, 0].set_xlim(xlim)
    ax2[0, 0].set_ylim(ylim_time)
    # Translation
    ax2[1, 0].boxplot(trans_err_eye, labels=outlier_count)
    ax2[1, 0].set_xlim(xlim)
    ax2[1, 0].set_ylim(ylim_iter)

    # EPnP
    rot_err_epnp = []
    trans_err_epnp = []
    for i in range(test_outlier):
        temp_rt = list_gnc_init[i]
        temp_rot = []
        temp_trans=[]
        for j in range(4):
            temp_rot.append(temp_rt[num_per_init+j].timing_GNC)
            temp_trans.append(temp_rt[num_per_init+j].iterations)
        rot_err_epnp.append(temp_rot)
        trans_err_epnp.append(temp_trans)
    ax2[0, 1].boxplot(rot_err_epnp, labels=outlier_count)
    ax2[0, 1].set_title("EPnP+GN")
    ax2[0, 1].set_xlim(xlim)
    ax2[0, 1].set_ylim(ylim_time)
    # Translation
    ax2[1, 1].boxplot(trans_err_epnp, labels=outlier_count)
    ax2[1, 1].set_xlim(xlim)
    ax2[1, 1].set_ylim(ylim_iter)

    # OpenCV-EPnP+GN
    rot_err_opencv = []
    trans_err_opencv = []
    for i in range(test_outlier):
        temp_rt = list_gnc_init[i]
        temp_rot = []
        temp_trans=[]
        for j in range(4):
            temp_rot.append(temp_rt[2*num_per_init+j].timing_GNC)
            temp_trans.append(temp_rt[2*num_per_init+j].iterations)
        rot_err_opencv.append(temp_rot)
        trans_err_opencv.append(temp_trans)
    ax2[0, 2].boxplot(rot_err_opencv, labels=outlier_count)
    ax2[0, 2].set_title("OpenCV EPnP+GN")
    ax2[0, 2].set_xlim(xlim)
    ax2[0, 2].set_ylim(ylim_time)

    ax2[1, 2].boxplot(trans_err_opencv, labels=outlier_count)
    ax2[1, 2].set_xlim(xlim)
    ax2[1, 2].set_ylim(ylim_iter)

    # Small Translation
    rot_err_trans_init = []
    trans_err_trans_init = []
    for i in range(test_outlier):
        temp_rt = list_gnc_init[i]
        temp_rot = []
        temp_trans=[]
        for j in range(4):
            temp_rot.append(temp_rt[3*num_per_init+j].timing_GNC)
            temp_trans.append(temp_rt[3*num_per_init+j].iterations)
        rot_err_trans_init.append(temp_rot)
        trans_err_trans_init.append(temp_trans)
    ax2[0, 3].boxplot(rot_err_trans_init, labels=outlier_count)
    ax2[0, 3].set_title("Small Translation")
    ax2[0, 3].set_xlim(xlim)
    ax2[0, 3].set_ylim(ylim_time)

    ax2[1, 3].boxplot(trans_err_trans_init, labels=outlier_count)
    ax2[1, 3].set_xlim(xlim)
    ax2[1, 3].set_ylim(ylim_iter)

    plt.setp(ax2[0, 0], ylabel='Time')
    plt.setp(ax2[1, 0], ylabel='Iterations')

    for i, a in enumerate(ax2):
        for j, b in enumerate(a):
            for n, label in enumerate(b.xaxis.get_ticklabels()):
                if (n) % 10 != 0:
                    label.set_visible(False)

    plt.show()

# Varying Number of Correspondences

## Varying Number of Correspondences - Test 1

In [ ]:
if varying_n == 1: 
    print(f"Varying Number of Correspondences")

    # Experiment Setup
    test_var_n = 200
    test_amount  = np.array([25, 50, 75])
    
    # Point Cloud
    ph_CAD_var = load_points_from_file(path_to_cad_car_06)
    ph_CAD_var = scale_points(ph_CAD_var, 10)
    

    # Transformation Matrices
    Tr_onlytrans = compute_T(
                    np.random.uniform(-np.pi, np.pi), 
                    np.random.uniform(-np.pi, np.pi), 
                    np.random.uniform(-np.pi, np.pi),    
                    np.random.uniform(-0.5,0.5),
                    np.random.uniform(-0.5,0.5),
                    np.random.uniform(-0.5,0.5)*3+10)

    # Camera Parameter Matrix
    focal = 800.
    u_0 = 320.
    v_0 = 240.
    Ca =  compute_C(focal, focal, u_0, v_0)

    # Test Loop
    list_var_corr_1 = []
    for i in trange(4, test_var_n):
        list_var_corr_1.append([])
        ph_CAD = downsample_points(ph_CAD_var, i)
        for j in test_amount:
            pix = compute_pixels(ph_CAD, Tr_onlytrans, Ca, sigma=5, outlier_percentage=j)
            pix, ph_CAD = shuffle_points(pix, ph_CAD)

            epnp = EPnP()
            epnp.load_data(ph_CAD, pix, Tr_onlytrans, Ca)
            epnp.define_gnc_parameters(eps = 1000)
            epnp.compute_epnp(GN = True, GNC=True)
            list_var_corr_1[i-4].append(epnp)

#### Plotting

In [ ]:
if varying_n == 1:
    fig, ax2 = plt.subplots(2,3, figsize=(12, 8))

    xlim = [4, test_var_n]
    ylim = [-5, 181]

    n_corr_count = [x for x in range(4, test_var_n)]

        # OpenCV RANSAC
    rot_err_25 = []
    rot_err_50 = []
    rot_err_75 = []
    trans_err_25 = []
    trans_err_50 = []
    trans_err_75 = []


    for i in range(len(list_var_corr_1)):
        # rot_err_25_2 = []
        # rot_err_50_2 = []
        # rot_err_75_2 = []
        # trans_err_25_2 = []
        # trans_err_50_2 = []
        # trans_err_75_2 = []
        temp1 = list_var_corr_1[i]
        rot_err_25.append(angular_distance_mat(temp1[0].T[:3,:3], temp1[0].Rt_best[:3,:3]))
        rot_err_50.append(angular_distance_mat(temp1[1].T[:3,:3], temp1[1].Rt_best[:3,:3]))
        rot_err_75.append(angular_distance_mat(temp1[2].T[:3,:3], temp1[2].Rt_best[:3,:3]))
        trans_err_25.append(translation_error(temp1[0].T[:3,3], temp1[0].Rt_best[:3,3])*10)
        trans_err_50.append(translation_error(temp1[1].T[:3,3], temp1[1].Rt_best[:3,3])*10)
        trans_err_75.append(translation_error(temp1[2].T[:3,3], temp1[2].Rt_best[:3,3])*10)

    ax2[0, 0].scatter(n_corr_count, rot_err_25)
    ax2[0, 0].set_title("25 percent outliers")
    ax2[0, 0].set_xlim(xlim)
    ax2[0, 0].set_ylim(ylim)
    ax2[1, 0].scatter(n_corr_count, trans_err_25)
    ax2[1, 0].set_xlim(xlim)
    ax2[1, 0].set_ylim(ylim)

    ax2[0, 1].scatter(n_corr_count, rot_err_50)
    ax2[0, 1].set_title("50 percent outliers")
    ax2[0, 1].set_xlim(xlim)
    ax2[0, 1].set_ylim(ylim)
    ax2[1, 1].scatter(n_corr_count, trans_err_50)
    ax2[1, 1].set_xlim(xlim)
    ax2[1, 1].set_ylim(ylim)

    ax2[0, 2].scatter(n_corr_count, rot_err_75)
    ax2[0, 2].set_title("75 percent outliers")
    ax2[0, 2].set_xlim(xlim)
    ax2[0, 2].set_ylim(ylim)
    ax2[1, 2].scatter(n_corr_count, trans_err_75)
    ax2[1, 2].set_xlim(xlim)
    ax2[1, 2].set_ylim(ylim)

    plt.setp(ax2[0, 0], ylabel='Rotation error')
    plt.setp(ax2[1, 0], ylabel='Translation error')

    plt.show()

## Varying Number of Correspondences - Test 2

In [ ]:
yoyoyoy = True
if yoyoyoy == True and varying_n == True:
    list_var_corr_2 = []
    j = 0
    for i in trange(25, 2001, 25):
        ph_CAD = downsample_points(ph_CAD_var, i)
        pix = compute_pixels(ph_CAD, Tr_onlytrans, Ca, sigma=5, outlier_percentage=50)
        pix, ph_CAD = shuffle_points(pix, ph_CAD)

        epnp = EPnP()
        epnp.load_data(ph_CAD, pix, Tr_onlytrans, Ca)
        epnp.define_gnc_parameters(eps = 1000)
        epnp.compute_epnp(GN = True, GNC=True)
        list_var_corr_2.append(epnp)

#### Plotting

In [ ]:
if yoyoyoy == True and varying_n == True:
    fig, ax2 = plt.subplots(1,2, figsize=(12, 6))

    xlim = [10, 2001]
    ylim = [-5, 181]

    n_corr_count = [x for x in range(25, 2001, 25)]

        # OpenCV RANSAC
    var_corr_test2_timing_gnc = []
    var_corr_test2_iterations_gnc = []

    for i in range(len(list_var_corr_2)):
        temp1 = list_var_corr_2[i]
        var_corr_test2_timing_gnc.append(temp1.timing_GNC)
        var_corr_test2_iterations_gnc.append(temp1.iterations)

    ax2[0].scatter(n_corr_count, var_corr_test2_timing_gnc)
    ax2[1].scatter(n_corr_count, var_corr_test2_iterations_gnc)
    # ax2[:].set_xlim(xlim)

    plt.setp(ax2[0], ylabel='Time (s)')
    plt.setp(ax2[1], ylabel='Number of Iterations')
    plt.setp(ax2[:], xlabel='Number of Correspondences')

    plt.show()